In [1]:
!python --version

Python 3.9.7


# Importing Libraries

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
mp_holistic = mp.solutions.holistic # Holistic 
mp_drawing = mp.solutions.drawing_utils # Drawing 

# Gathering Key Points

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                  
    results = model.process(image)                 
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) 
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) 
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 

In [6]:


def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 



In [7]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [8]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [9]:


pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)



In [10]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# Extracting Key Points

In [11]:
result_test = extract_keypoints(results)

In [12]:
result_test

array([ 0.57530701,  0.61537033, -1.50414526, ...,  0.        ,
        0.        ,  0.        ])

In [13]:
np.save('0', result_test)

In [14]:
np.load('0.npy')

array([ 0.57530701,  0.61537033, -1.50414526, ...,  0.        ,
        0.        ,  0.        ])

# Creating folders for Collection

In [15]:
DATA_PATH = os.path.join('MP_DATA') 

actions = np.array(['good morning', 'good evening','how are you'])

no_sequences = 20

sequence_length = 20

start_folder = 20

In [16]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass



# Collecting keypoints for training and testing

In [17]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

# Data Preprocessing

In [18]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [19]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [20]:
label_map = {label:num for num, label in enumerate(actions)}

In [21]:
label_map

{'good morning': 0, 'good evening': 1, 'how are you': 2}

In [22]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [23]:
np.array(sequences).shape

(60, 20, 1662)

In [24]:
np.array(labels).shape

(60,)

In [25]:
X = np.array(sequences)

In [26]:
X

array([[[ 0.52537775,  0.60765517, -1.34425271, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.56227887,  0.60666209, -1.23072386, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.57827514,  0.60279369, -1.02987349, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.57936537,  0.59979272, -1.18398023, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.58509177,  0.59935397, -1.20415902, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.59205878,  0.59698588, -1.21944451, ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.61973053,  0.59664077, -1.4201802 , ...,  0.58121848,
          0.62445647, -0.12078021],
        [ 0.60781562,  0.60080361, -1.43516648, ...,  0.54323065,
          0.93173164, -0.03023807],
        [ 0.59890395,  0.60199338, -1.40426838, ...,  0.54791486,
          0.94910008, -0.01176767],
        ...,
        [ 0.58233237,  0.56268334, -1.17621517, ...,  

In [27]:
y = to_categorical(labels).astype(int)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [29]:
y_test.shape

(3, 3)

# Creating Our LSTM model

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [31]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [38]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(20,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [39]:
res = [.7, 0.2, 0.1]

In [40]:
actions[np.argmax(res)]

'good morning'

In [41]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [42]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
2/2 [==============================] - 3s 40ms/step - loss: 1.1508 - categorical_accuracy: 0.3158
Epoch 2/2000
2/2 [==============================] - 0s 37ms/step - loss: 1.4821 - categorical_accuracy: 0.2105
Epoch 3/2000
2/2 [==============================] - 0s 34ms/step - loss: 3.4732 - categorical_accuracy: 0.2632
Epoch 4/2000
2/2 [==============================] - 0s 38ms/step - loss: 17.6246 - categorical_accuracy: 0.2982
Epoch 5/2000
2/2 [==============================] - 0s 36ms/step - loss: 10.9401 - categorical_accuracy: 0.2982
Epoch 6/2000
2/2 [==============================] - 0s 34ms/step - loss: 5.3921 - categorical_accuracy: 0.2807
Epoch 7/2000
2/2 [==============================] - 0s 35ms/step - loss: 2.6248 - categorical_accuracy: 0.3509
Epoch 8/2000
2/2 [==============================] - 0s 36ms/step - loss: 6.3413 - categorical_accuracy: 0.2982
Epoch 9/2000
2/2 [==============================] - 0s 36ms/step - loss: 4.6074 - categorical_accuracy: 0.2281

2/2 [==============================] - 0s 41ms/step - loss: 0.4965 - categorical_accuracy: 0.7193
Epoch 75/2000
2/2 [==============================] - 0s 41ms/step - loss: 0.4418 - categorical_accuracy: 0.7895
Epoch 76/2000
2/2 [==============================] - 0s 41ms/step - loss: 0.3661 - categorical_accuracy: 0.8772
Epoch 77/2000
2/2 [==============================] - 0s 38ms/step - loss: 0.3765 - categorical_accuracy: 0.8421
Epoch 78/2000
2/2 [==============================] - 0s 36ms/step - loss: 0.4056 - categorical_accuracy: 0.8421
Epoch 79/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.3409 - categorical_accuracy: 0.8947
Epoch 80/2000
2/2 [==============================] - 0s 36ms/step - loss: 0.3694 - categorical_accuracy: 0.8246
Epoch 81/2000
2/2 [==============================] - 0s 35ms/step - loss: 0.3104 - categorical_accuracy: 0.8421
Epoch 82/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.4530 - categorical_accuracy: 0.7719
Epoch 

2/2 [==============================] - 0s 36ms/step - loss: 0.9318 - categorical_accuracy: 0.7018
Epoch 147/2000
2/2 [==============================] - 0s 35ms/step - loss: 0.8362 - categorical_accuracy: 0.6491
Epoch 148/2000
2/2 [==============================] - 0s 36ms/step - loss: 0.9994 - categorical_accuracy: 0.5614
Epoch 149/2000
2/2 [==============================] - 0s 38ms/step - loss: 1.2860 - categorical_accuracy: 0.4912
Epoch 150/2000
2/2 [==============================] - 0s 36ms/step - loss: 0.5646 - categorical_accuracy: 0.6316
Epoch 151/2000
2/2 [==============================] - 0s 35ms/step - loss: 0.6536 - categorical_accuracy: 0.6667
Epoch 152/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.4708 - categorical_accuracy: 0.7719
Epoch 153/2000
2/2 [==============================] - 0s 36ms/step - loss: 0.3509 - categorical_accuracy: 0.9123
Epoch 154/2000
2/2 [==============================] - 0s 36ms/step - loss: 0.3683 - categorical_accuracy: 0.824

2/2 [==============================] - 0s 36ms/step - loss: 0.1322 - categorical_accuracy: 0.9474
Epoch 219/2000
2/2 [==============================] - 0s 36ms/step - loss: 0.1108 - categorical_accuracy: 0.9474
Epoch 220/2000
2/2 [==============================] - 0s 33ms/step - loss: 0.0743 - categorical_accuracy: 0.9825
Epoch 221/2000
2/2 [==============================] - 0s 37ms/step - loss: 0.0730 - categorical_accuracy: 0.9649
Epoch 222/2000
2/2 [==============================] - 0s 37ms/step - loss: 0.0550 - categorical_accuracy: 0.9825
Epoch 223/2000
2/2 [==============================] - 0s 35ms/step - loss: 0.0695 - categorical_accuracy: 0.9474
Epoch 224/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.0515 - categorical_accuracy: 0.9825
Epoch 225/2000
2/2 [==============================] - 0s 35ms/step - loss: 0.0488 - categorical_accuracy: 0.9825
Epoch 226/2000
2/2 [==============================] - 0s 36ms/step - loss: 0.0537 - categorical_accuracy: 0.964

2/2 [==============================] - 0s 40ms/step - loss: 9.9981e-04 - categorical_accuracy: 1.0000
Epoch 291/2000
2/2 [==============================] - 0s 39ms/step - loss: 9.8980e-04 - categorical_accuracy: 1.0000
Epoch 292/2000
2/2 [==============================] - 0s 37ms/step - loss: 9.8734e-04 - categorical_accuracy: 1.0000
Epoch 293/2000
2/2 [==============================] - 0s 39ms/step - loss: 9.6064e-04 - categorical_accuracy: 1.0000
Epoch 294/2000
2/2 [==============================] - 0s 38ms/step - loss: 9.2276e-04 - categorical_accuracy: 1.0000
Epoch 295/2000
2/2 [==============================] - 0s 44ms/step - loss: 9.0060e-04 - categorical_accuracy: 1.0000
Epoch 296/2000
2/2 [==============================] - 0s 40ms/step - loss: 8.8067e-04 - categorical_accuracy: 1.0000
Epoch 297/2000
2/2 [==============================] - 0s 41ms/step - loss: 8.6377e-04 - categorical_accuracy: 1.0000
Epoch 298/2000
2/2 [==============================] - 0s 41ms/step - loss: 8.62

...

2/2 [==============================] - 0s 42ms/step - loss: 7.5833e-06 - categorical_accuracy: 1.0000
Epoch 1681/2000
2/2 [==============================] - 0s 40ms/step - loss: 7.5477e-06 - categorical_accuracy: 1.0000
Epoch 1682/2000
2/2 [==============================] - 0s 40ms/step - loss: 7.5268e-06 - categorical_accuracy: 1.0000
Epoch 1683/2000
2/2 [==============================] - 0s 51ms/step - loss: 7.4850e-06 - categorical_accuracy: 1.0000
Epoch 1684/2000
2/2 [==============================] - 0s 45ms/step - loss: 7.4536e-06 - categorical_accuracy: 1.0000
Epoch 1685/2000
2/2 [==============================] - 0s 43ms/step - loss: 7.4076e-06 - categorical_accuracy: 1.0000
Epoch 1686/2000
2/2 [==============================] - 0s 43ms/step - loss: 7.3658e-06 - categorical_accuracy: 1.0000
Epoch 1687/2000
2/2 [==============================] - 0s 43ms/step - loss: 7.3469e-06 - categorical_accuracy: 1.0000
Epoch 1688/2000
2/2 [==============================] - 0s 41ms/step - lo

2/2 [==============================] - 0s 42ms/step - loss: 4.5174e-06 - categorical_accuracy: 1.0000
Epoch 1750/2000
2/2 [==============================] - 0s 47ms/step - loss: 4.4567e-06 - categorical_accuracy: 1.0000
Epoch 1751/2000
2/2 [==============================] - 0s 42ms/step - loss: 4.3772e-06 - categorical_accuracy: 1.0000
Epoch 1752/2000
2/2 [==============================] - 0s 44ms/step - loss: 4.3291e-06 - categorical_accuracy: 1.0000
Epoch 1753/2000
2/2 [==============================] - 0s 42ms/step - loss: 4.2476e-06 - categorical_accuracy: 1.0000
Epoch 1754/2000
2/2 [==============================] - 0s 42ms/step - loss: 4.1765e-06 - categorical_accuracy: 1.0000
Epoch 1755/2000
2/2 [==============================] - 0s 42ms/step - loss: 4.1095e-06 - categorical_accuracy: 1.0000
Epoch 1756/2000
2/2 [==============================] - 0s 39ms/step - loss: 4.0322e-06 - categorical_accuracy: 1.0000
Epoch 1757/2000
2/2 [==============================] - 0s 38ms/step - lo

2/2 [==============================] - 0s 45ms/step - loss: 1.3803e-06 - categorical_accuracy: 1.0000
Epoch 1819/2000
2/2 [==============================] - 0s 42ms/step - loss: 1.3719e-06 - categorical_accuracy: 1.0000
Epoch 1820/2000
2/2 [==============================] - 0s 37ms/step - loss: 1.3531e-06 - categorical_accuracy: 1.0000
Epoch 1821/2000
2/2 [==============================] - 0s 38ms/step - loss: 1.3427e-06 - categorical_accuracy: 1.0000
Epoch 1822/2000
2/2 [==============================] - 0s 39ms/step - loss: 1.3259e-06 - categorical_accuracy: 1.0000
Epoch 1823/2000
2/2 [==============================] - 0s 46ms/step - loss: 1.3134e-06 - categorical_accuracy: 1.0000
Epoch 1824/2000
2/2 [==============================] - 0s 39ms/step - loss: 1.3008e-06 - categorical_accuracy: 1.0000
Epoch 1825/2000
2/2 [==============================] - 0s 37ms/step - loss: 1.2841e-06 - categorical_accuracy: 1.0000
Epoch 1826/2000
2/2 [==============================] - 0s 37ms/step - lo

2/2 [==============================] - 0s 45ms/step - loss: 7.7799e-07 - categorical_accuracy: 1.0000
Epoch 1888/2000
2/2 [==============================] - 0s 43ms/step - loss: 7.7590e-07 - categorical_accuracy: 1.0000
Epoch 1889/2000
2/2 [==============================] - 0s 49ms/step - loss: 7.7172e-07 - categorical_accuracy: 1.0000
Epoch 1890/2000
2/2 [==============================] - 0s 42ms/step - loss: 7.6754e-07 - categorical_accuracy: 1.0000
Epoch 1891/2000
2/2 [==============================] - 0s 42ms/step - loss: 7.5917e-07 - categorical_accuracy: 1.0000
Epoch 1892/2000
2/2 [==============================] - 0s 42ms/step - loss: 7.5708e-07 - categorical_accuracy: 1.0000
Epoch 1893/2000
2/2 [==============================] - 0s 41ms/step - loss: 7.4872e-07 - categorical_accuracy: 1.0000
Epoch 1894/2000
2/2 [==============================] - 0s 42ms/step - loss: 7.4662e-07 - categorical_accuracy: 1.0000
Epoch 1895/2000
2/2 [==============================] - 0s 43ms/step - lo

2/2 [==============================] - 0s 40ms/step - loss: 5.8977e-07 - categorical_accuracy: 1.0000
Epoch 1957/2000
2/2 [==============================] - 0s 42ms/step - loss: 5.8977e-07 - categorical_accuracy: 1.0000
Epoch 1958/2000
2/2 [==============================] - 0s 41ms/step - loss: 5.8350e-07 - categorical_accuracy: 1.0000
Epoch 1959/2000
2/2 [==============================] - 0s 43ms/step - loss: 5.8141e-07 - categorical_accuracy: 1.0000
Epoch 1960/2000
2/2 [==============================] - 0s 44ms/step - loss: 5.8141e-07 - categorical_accuracy: 1.0000
Epoch 1961/2000
2/2 [==============================] - 0s 46ms/step - loss: 5.7931e-07 - categorical_accuracy: 1.0000
Epoch 1962/2000
2/2 [==============================] - 0s 43ms/step - loss: 5.7931e-07 - categorical_accuracy: 1.0000
Epoch 1963/2000
2/2 [==============================] - 0s 44ms/step - loss: 5.7722e-07 - categorical_accuracy: 1.0000
Epoch 1964/2000
2/2 [==============================] - 0s 46ms/step - lo

In [43]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 20, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 20, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-tr

In [44]:
res = model.predict(X_test)

1/1 [==============================] - 0s 309ms/step


# Making prediction accuracy score

In [45]:
actions[np.argmax(res[1])]

'how are you'

In [46]:
model.save('action.h5')

In [ ]:
#pipi = tf.saved_model.load('action.h5')

In [47]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from tensorflow.keras.models import load_model

In [ ]:
#pipi = load_model('action.h5')

In [48]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 19ms/step


In [49]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [50]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[1, 0],
        [2, 0]],

       [[1, 1],
        [0, 1]],

       [[2, 1],
        [0, 0]]], dtype=int64)

In [52]:
accuracy_score(ytrue, yhat)

0.3333333333333333

# Test in real time

In [53]:


colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame



In [55]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 15ms/step
good morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 14ms/step
good evening
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 14ms/step
good morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
good morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 14ms/step
good morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
good morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
good morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
good morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [===================

1/1 [==============================] - 0s 15ms/step
good morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
good morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
good morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
good morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
good morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
good morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
good morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
good morning
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [===================